# Preambolo
prova a considerare class weight balanced.
Fai albero ottimo 3 vs altri, 4 vs altri e fai il pdf-->10 iterazioni, ricordati di fare holdout
Per classe 4  fai split 70-30.
Tumori maligni --> 1 carcinoma, 2 linfoma , 5 è cisto adenoma ma non se se de preciso così e se è benigno o maligno.
Per ora 5 lo considero poco, è una classe un po' indeterminata

1,2 vs 3,4,5--> maligni contro benigni

f1-score--> meglio dell'accuratezza per classi non bilanciate
tu cerca di aggirare lo sbolanciamento tra classi--> nell'allenamento degli alberi considero lo weight pesando più le classi meno rappresentate-->peso inversamente proporzionale al numero

nella media dell'accuratezza visualizza anche dev std


## Moduli importati

In [1]:
import graphviz
import pandas as pd
import numpy as np
from imblearn.over_sampling import SMOTENC, SMOTE
from sklearn import tree
from sklearn.model_selection import GridSearchCV, train_test_split, cross_val_score, StratifiedKFold
from sklearn.metrics import classification_report, accuracy_score, f1_score, make_scorer, precision_score

## Importo dataset

In [2]:
df = pd.read_excel('../Db_parotide_Def_REV_Samuel.xlsx')
df['ADC'] = df['ADC'].apply(lambda s: str(s).replace(',', '.')).astype(float)
df.iat[0, 6] = 3
df.iat[1, 6] = 3
df.iat[20, 6] = 3
df.head()

,ID,eta,Segni macro malignità,ADC,TIC type,T2,Output Algoritmo,COD ISTO,ISTO DEFINITIVO-PULITO,ISTO DEFINITIVO,...,CONDORD CODICE FNAC-ISTO DIAGN,CONCORD multiRMN-ISTO B/M,CONCORD multiRMN-ISTO DIAGN,CONCORD FNAC-RMN,FNAC,DATA FNAC,ESITO FNAC,COD FNAC,MILAN,DATA INTERVENTO
0,4,33,0,2.60,D,0,3,1,CARCINOMA,CARCINOMA SQUAMOCELLULARE G2,...,1,1,1,1,0,27/12/2017,Positivo per CTM. Carcinoma squamocellulare o ...,3,6,24/01/2018
1,10,50,0,2.44,D,0,3,1,CARCINOMA,CARCINOMA ADENOIDECISTICO G3,...,1,1,1,1,0,27/12/2016,carcinoma adenoidecistico,3,6,27/11/2019
2,9,81,1,0.70,C,1,1,1,CARCINOMA,CARCINOMA SQUAMOCELLULARE G2,...,1,1,1,1,0,21/03/2019,"Positivo per CTM, carcinoma squamocellulare",3,6,15/05/2019
3,1,28,0,1.60,A,1,3,1,CARCINOMA,CARCINOMA DEI DOTTI SALIVARI,...,1,1,1,1,0,2016-06-10 00:00:00,sospetto per CTM,5,5,25/01/2017
4,7,51,0,1.40,A,0,3,1,METASTASI DA CARCINOMA SQUAMOCELLULARE ALLA PA...,METASTASI DA CARCINOMA SQUAMOCELLULARE ALLA PA...,...,1,1,1,1,0,13/09/2018,Positivo per CTM. Carcinoma squamocellulare,3,6,2018-10-10 00:00:00


## Samples e labels

In [3]:
X_not_encoded = df.loc[:,'eta':'T2'].copy()
X = pd.get_dummies(X_not_encoded, columns = ['TIC type'])
y = df['COD ISTO'].copy()
y.value_counts()


3    54
4    27
1    16
2     5
5     5
Name: COD ISTO, dtype: int64

## Funzione

In [4]:
#qui faccio iterazioni di stratified k fold, splitto stratificando, augmento il training set, faccio augmentation con il training set
#prima di rendere le labels binarie, calcolo una score sul test set e alla fine restituisco la media delle score 
#con la deviaizione std


#assumo di passargli X not encode e y
#se scegli smote e non smote sarò io a fare la one hot encoding 

def AUG(X, y, target_one, tuned_parameters, aug_method = 'smotenc',
                scoring_func = accuracy_score, n_splits= 5,
                random_state_tree = None,
                n_it = 10, filename = 'albero'):

    scoring = make_scorer(scoring_func)
    avg_scores = []
    models_ = []
    
    for n in range(n_it):
    
        print('### ITERATION %d ###\n\n' %(n+1))

        scores = []
        models = []

        skf = StratifiedKFold(n_splits = n_splits, shuffle = True, random_state = None)

        for train_index, test_index in skf.split(X, y):

            X_train, X_test = X.iloc[train_index], X.iloc[test_index]
            y_train, y_test = y.iloc[train_index], y.iloc[test_index]

            #sempre che con smote faccio oversampling di tutte le classi non maggioritarie 
            if aug_method == 'smotenc':#assumo tutte features categoriche 
                sm = SMOTENC(random_state = None, k_neighbors = 3,
                                                   categorical_features = [3])
                X_train_aug, y_train_aug = sm.fit_resample(X_train, y_train)
                X_train_aug = pd.get_dummies(X_train_aug, columns = ['TIC type'])
                
                

            elif aug_method == 'smote':#devo fare one hot encoding 
                X_train = pd.get_dummies(X_train, columns = ['TIC type'])
                sm = SMOTE(random_state = None, k_neighbors = 3)
                X_train_aug, y_train_aug = sm.fit_resample(X_train, y_train)
            

            y_train_aug_bin = [1 if x in target_one else 0 for x in y_train_aug]
            y_test_bin = [1 if x in target_one else 0 for x in y_test]
            X_test = pd.get_dummies(X_test, columns = ['TIC type'])
            
            if 'TIC type_A' not in X_train_aug.columns:
                X_train_aug['TIC type_A'] = 0
            if 'TIC type_B' not in X_train_aug.columns:
                X_train_aug['TIC type_B'] = 0
            if 'TIC type_C' not in X_train_aug.columns:
                X_train_aug['TIC type_C'] = 0
            if 'TIC type_D' not in X_train_aug.columns:
                X_train_aug['TIC type_D'] = 0
            
            if 'TIC type_A' not in X_test.columns:
                X_test['TIC type_A'] = 0
            if 'TIC type_B' not in X_test.columns:
                X_test['TIC type_B'] = 0
            if 'TIC type_C' not in X_test.columns:
                X_test['TIC type_C'] = 0
            if 'TIC type_D' not in X_test.columns:
                X_test['TIC type_D'] = 0

            albero = tree.DecisionTreeClassifier(random_state = random_state_tree)

            #la famo la gridsearch?? Si daje
            clf = GridSearchCV(albero, tuned_parameters, scoring = scoring,
                               cv = StratifiedKFold(n_splits = 2, shuffle = True, random_state = None),
                               refit = True)
            clf.fit(X_train_aug, y_train_aug_bin)
            
            models.append(clf.best_estimator_)

            params = clf.best_params_
            score = clf.best_score_


            print("Parameters set found:\n")
            print(params)
            print()
            print("Mean cross-validated score of the estimator: %0.3f\n" %(score))
            print()
            current_score = scoring_func(y_test_bin, clf.predict(X_test))
            print('Score computed on the test set: %0.3f\n' %(current_score))
            print('-'*100)

            scores.append(current_score)
            
        avg_scores.append(np.mean(scores))
        models_.append(models[np.argmax(scores)])
        
    print('\n\n# Final Mean score: %0.3f\n# Std dev: %0.3f' %(np.mean(avg_scores), np.std(avg_scores)))
    
    best_model = models_[np.argmax(avg_scores)]
    
    data = tree.export_graphviz(best_model, out_file=None, 
                      feature_names=X_train_aug.columns,  
                      class_names=['0','1'],  
                      filled=True, rounded=True,  
                      special_characters=True) 
    graph = graphviz.Source(data) 
    graph.render('SMOTE-'+filename)
    

In [5]:
seed = 42 
tuned_parameters = {'criterion': ['gini', 'entropy'],
                    'max_depth': list(range(2,11)),     
                    'min_samples_leaf': list(range(2,11)),
                    'class_weight': ['balanced', None]}

In [6]:
AUG(X_not_encoded, y, [3], tuned_parameters, random_state_tree = seed,
    scoring_func = accuracy_score, filename = '3-acc', aug_method = 'smotenc')

### ITERATION 1 ###


Parameters set found:

{'class_weight': None, 'criterion': 'gini', 'max_depth': 2, 'min_samples_leaf': 4}

Mean cross-validated score of the estimator: 0.836


Score computed on the test set: 0.636

----------------------------------------------------------------------------------------------------
Parameters set found:

{'class_weight': None, 'criterion': 'gini', 'max_depth': 2, 'min_samples_leaf': 4}

Mean cross-validated score of the estimator: 0.800


Score computed on the test set: 0.636

----------------------------------------------------------------------------------------------------
Parameters set found:

{'class_weight': None, 'criterion': 'gini', 'max_depth': 4, 'min_samples_leaf': 6}

Mean cross-validated score of the estimator: 0.860


Score computed on the test set: 0.476

----------------------------------------------------------------------------------------------------
Parameters set found:

{'class_weight': None, 'criterion': 'entropy', 'max_dep

Parameters set found:

{'class_weight': None, 'criterion': 'gini', 'max_depth': 3, 'min_samples_leaf': 3}

Mean cross-validated score of the estimator: 0.828


Score computed on the test set: 0.476

----------------------------------------------------------------------------------------------------
Parameters set found:

{'class_weight': None, 'criterion': 'gini', 'max_depth': 3, 'min_samples_leaf': 5}

Mean cross-validated score of the estimator: 0.818


Score computed on the test set: 0.476

----------------------------------------------------------------------------------------------------
Parameters set found:

{'class_weight': None, 'criterion': 'gini', 'max_depth': 4, 'min_samples_leaf': 3}

Mean cross-validated score of the estimator: 0.828


Score computed on the test set: 0.524

----------------------------------------------------------------------------------------------------
### ITERATION 7 ###


Parameters set found:

{'class_weight': None, 'criterion': 'gini', 'max_depth'

In [7]:
AUG(X_not_encoded, y, [3], tuned_parameters, random_state_tree = seed,
    scoring_func = f1_score, filename = '3-f1', aug_method = 'smotenc')

### ITERATION 1 ###


Parameters set found:

{'class_weight': 'balanced', 'criterion': 'gini', 'max_depth': 5, 'min_samples_leaf': 3}

Mean cross-validated score of the estimator: 0.506


Score computed on the test set: 0.435

----------------------------------------------------------------------------------------------------
Parameters set found:

{'class_weight': 'balanced', 'criterion': 'gini', 'max_depth': 7, 'min_samples_leaf': 2}

Mean cross-validated score of the estimator: 0.541


Score computed on the test set: 0.316

----------------------------------------------------------------------------------------------------
Parameters set found:

{'class_weight': 'balanced', 'criterion': 'entropy', 'max_depth': 4, 'min_samples_leaf': 2}

Mean cross-validated score of the estimator: 0.567


Score computed on the test set: 0.556

----------------------------------------------------------------------------------------------------
Parameters set found:

{'class_weight': 'balanced', 'crit

Parameters set found:

{'class_weight': 'balanced', 'criterion': 'gini', 'max_depth': 8, 'min_samples_leaf': 2}

Mean cross-validated score of the estimator: 0.549


Score computed on the test set: 0.480

----------------------------------------------------------------------------------------------------
Parameters set found:

{'class_weight': None, 'criterion': 'gini', 'max_depth': 3, 'min_samples_leaf': 9}

Mean cross-validated score of the estimator: 0.454


Score computed on the test set: 0.571

----------------------------------------------------------------------------------------------------
Parameters set found:

{'class_weight': 'balanced', 'criterion': 'gini', 'max_depth': 2, 'min_samples_leaf': 10}

Mean cross-validated score of the estimator: 0.538


Score computed on the test set: 0.560

----------------------------------------------------------------------------------------------------
### ITERATION 7 ###


Parameters set found:

{'class_weight': 'balanced', 'criterion': 

In [8]:
AUG(X_not_encoded, y, [3], tuned_parameters, random_state_tree = seed,
    scoring_func = accuracy_score, filename = '3-acc-smote', aug_method = 'smote')

### ITERATION 1 ###


Parameters set found:

{'class_weight': None, 'criterion': 'entropy', 'max_depth': 4, 'min_samples_leaf': 9}

Mean cross-validated score of the estimator: 0.827


Score computed on the test set: 0.545

----------------------------------------------------------------------------------------------------
Parameters set found:

{'class_weight': None, 'criterion': 'entropy', 'max_depth': 3, 'min_samples_leaf': 2}

Mean cross-validated score of the estimator: 0.828


Score computed on the test set: 0.591

----------------------------------------------------------------------------------------------------
Parameters set found:

{'class_weight': None, 'criterion': 'entropy', 'max_depth': 7, 'min_samples_leaf': 2}

Mean cross-validated score of the estimator: 0.888


Score computed on the test set: 0.524

----------------------------------------------------------------------------------------------------
Parameters set found:

{'class_weight': None, 'criterion': 'gini', 'm

Parameters set found:

{'class_weight': 'balanced', 'criterion': 'gini', 'max_depth': 3, 'min_samples_leaf': 2}

Mean cross-validated score of the estimator: 0.842


Score computed on the test set: 0.476

----------------------------------------------------------------------------------------------------
Parameters set found:

{'class_weight': None, 'criterion': 'gini', 'max_depth': 8, 'min_samples_leaf': 2}

Mean cross-validated score of the estimator: 0.828


Score computed on the test set: 0.524

----------------------------------------------------------------------------------------------------
Parameters set found:

{'class_weight': 'balanced', 'criterion': 'gini', 'max_depth': 5, 'min_samples_leaf': 2}

Mean cross-validated score of the estimator: 0.809


Score computed on the test set: 0.524

----------------------------------------------------------------------------------------------------
### ITERATION 7 ###


Parameters set found:

{'class_weight': None, 'criterion': 'gini',

In [9]:
AUG(X_not_encoded, y, [3], tuned_parameters, random_state_tree = seed,
    scoring_func = f1_score, filename = '3-f1-smote', aug_method = 'smote')

### ITERATION 1 ###


Parameters set found:

{'class_weight': None, 'criterion': 'entropy', 'max_depth': 4, 'min_samples_leaf': 3}

Mean cross-validated score of the estimator: 0.666


Score computed on the test set: 0.125

----------------------------------------------------------------------------------------------------
Parameters set found:

{'class_weight': 'balanced', 'criterion': 'entropy', 'max_depth': 4, 'min_samples_leaf': 6}

Mean cross-validated score of the estimator: 0.586


Score computed on the test set: 0.615

----------------------------------------------------------------------------------------------------
Parameters set found:

{'class_weight': 'balanced', 'criterion': 'entropy', 'max_depth': 2, 'min_samples_leaf': 2}

Mean cross-validated score of the estimator: 0.569


Score computed on the test set: 0.571

----------------------------------------------------------------------------------------------------
Parameters set found:

{'class_weight': 'balanced', 'crit

Parameters set found:

{'class_weight': None, 'criterion': 'gini', 'max_depth': 8, 'min_samples_leaf': 2}

Mean cross-validated score of the estimator: 0.522


Score computed on the test set: 0.421

----------------------------------------------------------------------------------------------------
Parameters set found:

{'class_weight': 'balanced', 'criterion': 'gini', 'max_depth': 2, 'min_samples_leaf': 3}

Mean cross-validated score of the estimator: 0.577


Score computed on the test set: 0.667

----------------------------------------------------------------------------------------------------
Parameters set found:

{'class_weight': 'balanced', 'criterion': 'entropy', 'max_depth': 6, 'min_samples_leaf': 3}

Mean cross-validated score of the estimator: 0.510


Score computed on the test set: 0.571

----------------------------------------------------------------------------------------------------
### ITERATION 7 ###


Parameters set found:

{'class_weight': 'balanced', 'criterion'

In [10]:
AUG(X_not_encoded, y, [4], tuned_parameters, random_state_tree = seed,
    scoring_func = accuracy_score, filename = '4-acc', aug_method = 'smotenc')

### ITERATION 1 ###


Parameters set found:

{'class_weight': None, 'criterion': 'entropy', 'max_depth': 4, 'min_samples_leaf': 7}

Mean cross-validated score of the estimator: 0.832


Score computed on the test set: 0.682

----------------------------------------------------------------------------------------------------
Parameters set found:

{'class_weight': None, 'criterion': 'entropy', 'max_depth': 6, 'min_samples_leaf': 4}

Mean cross-validated score of the estimator: 0.823


Score computed on the test set: 0.773

----------------------------------------------------------------------------------------------------
Parameters set found:

{'class_weight': None, 'criterion': 'gini', 'max_depth': 4, 'min_samples_leaf': 4}

Mean cross-validated score of the estimator: 0.814


Score computed on the test set: 0.714

----------------------------------------------------------------------------------------------------
Parameters set found:

{'class_weight': None, 'criterion': 'entropy', 'm

Parameters set found:

{'class_weight': None, 'criterion': 'gini', 'max_depth': 2, 'min_samples_leaf': 9}

Mean cross-validated score of the estimator: 0.800


Score computed on the test set: 0.762

----------------------------------------------------------------------------------------------------
Parameters set found:

{'class_weight': None, 'criterion': 'gini', 'max_depth': 6, 'min_samples_leaf': 2}

Mean cross-validated score of the estimator: 0.870


Score computed on the test set: 0.667

----------------------------------------------------------------------------------------------------
Parameters set found:

{'class_weight': None, 'criterion': 'gini', 'max_depth': 2, 'min_samples_leaf': 9}

Mean cross-validated score of the estimator: 0.828


Score computed on the test set: 0.714

----------------------------------------------------------------------------------------------------
### ITERATION 7 ###


Parameters set found:

{'class_weight': None, 'criterion': 'entropy', 'max_dep

In [11]:
AUG(X_not_encoded, y, [4], tuned_parameters, random_state_tree = seed,
    scoring_func = f1_score, filename = '4-f1', aug_method = 'smotenc')

### ITERATION 1 ###


Parameters set found:

{'class_weight': 'balanced', 'criterion': 'entropy', 'max_depth': 5, 'min_samples_leaf': 2}

Mean cross-validated score of the estimator: 0.467


Score computed on the test set: 0.316

----------------------------------------------------------------------------------------------------
Parameters set found:

{'class_weight': 'balanced', 'criterion': 'entropy', 'max_depth': 7, 'min_samples_leaf': 3}

Mean cross-validated score of the estimator: 0.486


Score computed on the test set: 0.421

----------------------------------------------------------------------------------------------------
Parameters set found:

{'class_weight': 'balanced', 'criterion': 'entropy', 'max_depth': 4, 'min_samples_leaf': 9}

Mean cross-validated score of the estimator: 0.512


Score computed on the test set: 0.222

----------------------------------------------------------------------------------------------------
Parameters set found:

{'class_weight': None, 'crit

Parameters set found:

{'class_weight': 'balanced', 'criterion': 'entropy', 'max_depth': 7, 'min_samples_leaf': 2}

Mean cross-validated score of the estimator: 0.505


Score computed on the test set: 0.400

----------------------------------------------------------------------------------------------------
Parameters set found:

{'class_weight': 'balanced', 'criterion': 'gini', 'max_depth': 5, 'min_samples_leaf': 2}

Mean cross-validated score of the estimator: 0.593


Score computed on the test set: 0.571

----------------------------------------------------------------------------------------------------
Parameters set found:

{'class_weight': 'balanced', 'criterion': 'gini', 'max_depth': 4, 'min_samples_leaf': 6}

Mean cross-validated score of the estimator: 0.519


Score computed on the test set: 0.333

----------------------------------------------------------------------------------------------------
### ITERATION 7 ###


Parameters set found:

{'class_weight': 'balanced', 'crit

In [12]:
AUG(X_not_encoded, y, [4], tuned_parameters, random_state_tree = seed,
    scoring_func = accuracy_score, filename = '4-acc-smote', aug_method = 'smote')

### ITERATION 1 ###


Parameters set found:

{'class_weight': None, 'criterion': 'gini', 'max_depth': 2, 'min_samples_leaf': 3}

Mean cross-validated score of the estimator: 0.805


Score computed on the test set: 0.682

----------------------------------------------------------------------------------------------------
Parameters set found:

{'class_weight': None, 'criterion': 'gini', 'max_depth': 2, 'min_samples_leaf': 5}

Mean cross-validated score of the estimator: 0.805


Score computed on the test set: 0.727

----------------------------------------------------------------------------------------------------
Parameters set found:

{'class_weight': None, 'criterion': 'gini', 'max_depth': 5, 'min_samples_leaf': 2}

Mean cross-validated score of the estimator: 0.809


Score computed on the test set: 0.619

----------------------------------------------------------------------------------------------------
Parameters set found:

{'class_weight': None, 'criterion': 'entropy', 'max_dep

Parameters set found:

{'class_weight': None, 'criterion': 'gini', 'max_depth': 6, 'min_samples_leaf': 2}

Mean cross-validated score of the estimator: 0.833


Score computed on the test set: 0.571

----------------------------------------------------------------------------------------------------
Parameters set found:

{'class_weight': None, 'criterion': 'entropy', 'max_depth': 8, 'min_samples_leaf': 2}

Mean cross-validated score of the estimator: 0.809


Score computed on the test set: 0.714

----------------------------------------------------------------------------------------------------
Parameters set found:

{'class_weight': None, 'criterion': 'gini', 'max_depth': 9, 'min_samples_leaf': 2}

Mean cross-validated score of the estimator: 0.809


Score computed on the test set: 0.714

----------------------------------------------------------------------------------------------------
### ITERATION 7 ###


Parameters set found:

{'class_weight': None, 'criterion': 'gini', 'max_dep

In [13]:
AUG(X_not_encoded, y, [4], tuned_parameters, random_state_tree = seed,
    scoring_func = f1_score, filename = '4-f1-smote', aug_method = 'smote')

### ITERATION 1 ###


Parameters set found:

{'class_weight': 'balanced', 'criterion': 'entropy', 'max_depth': 7, 'min_samples_leaf': 5}

Mean cross-validated score of the estimator: 0.416


Score computed on the test set: 0.267

----------------------------------------------------------------------------------------------------
Parameters set found:

{'class_weight': None, 'criterion': 'entropy', 'max_depth': 6, 'min_samples_leaf': 5}

Mean cross-validated score of the estimator: 0.491


Score computed on the test set: 0.250

----------------------------------------------------------------------------------------------------
Parameters set found:

{'class_weight': 'balanced', 'criterion': 'gini', 'max_depth': 9, 'min_samples_leaf': 2}

Mean cross-validated score of the estimator: 0.424


Score computed on the test set: 0.167

----------------------------------------------------------------------------------------------------
Parameters set found:

{'class_weight': 'balanced', 'criteri

Parameters set found:

{'class_weight': None, 'criterion': 'gini', 'max_depth': 4, 'min_samples_leaf': 4}

Mean cross-validated score of the estimator: 0.458


Score computed on the test set: 0.400

----------------------------------------------------------------------------------------------------
Parameters set found:

{'class_weight': None, 'criterion': 'gini', 'max_depth': 6, 'min_samples_leaf': 3}

Mean cross-validated score of the estimator: 0.640


Score computed on the test set: 0.250

----------------------------------------------------------------------------------------------------
Parameters set found:

{'class_weight': 'balanced', 'criterion': 'gini', 'max_depth': 5, 'min_samples_leaf': 8}

Mean cross-validated score of the estimator: 0.414


Score computed on the test set: 0.111

----------------------------------------------------------------------------------------------------
### ITERATION 7 ###


Parameters set found:

{'class_weight': None, 'criterion': 'entropy', 'm

In [14]:
AUG(X_not_encoded, y, [1,2], tuned_parameters, random_state_tree = seed,
    scoring_func = accuracy_score, filename = '12-acc', aug_method = 'smotenc')

### ITERATION 1 ###


Parameters set found:

{'class_weight': None, 'criterion': 'entropy', 'max_depth': 6, 'min_samples_leaf': 3}

Mean cross-validated score of the estimator: 0.759


Score computed on the test set: 0.500

----------------------------------------------------------------------------------------------------
Parameters set found:

{'class_weight': None, 'criterion': 'gini', 'max_depth': 5, 'min_samples_leaf': 2}

Mean cross-validated score of the estimator: 0.763


Score computed on the test set: 0.636

----------------------------------------------------------------------------------------------------
Parameters set found:

{'class_weight': None, 'criterion': 'entropy', 'max_depth': 4, 'min_samples_leaf': 2}

Mean cross-validated score of the estimator: 0.684


Score computed on the test set: 0.857

----------------------------------------------------------------------------------------------------
Parameters set found:

{'class_weight': None, 'criterion': 'entropy', 'm

Parameters set found:

{'class_weight': None, 'criterion': 'gini', 'max_depth': 6, 'min_samples_leaf': 6}

Mean cross-validated score of the estimator: 0.739


Score computed on the test set: 0.524

----------------------------------------------------------------------------------------------------
Parameters set found:

{'class_weight': None, 'criterion': 'entropy', 'max_depth': 8, 'min_samples_leaf': 4}

Mean cross-validated score of the estimator: 0.721


Score computed on the test set: 0.810

----------------------------------------------------------------------------------------------------
Parameters set found:

{'class_weight': 'balanced', 'criterion': 'entropy', 'max_depth': 10, 'min_samples_leaf': 2}

Mean cross-validated score of the estimator: 0.679


Score computed on the test set: 0.476

----------------------------------------------------------------------------------------------------
### ITERATION 7 ###


Parameters set found:

{'class_weight': None, 'criterion': 'entro

In [15]:
AUG(X_not_encoded, y, [1,2], tuned_parameters, random_state_tree = seed,
    scoring_func = f1_score, filename = '12-f1', aug_method = 'smotenc')

### ITERATION 1 ###


Parameters set found:

{'class_weight': 'balanced', 'criterion': 'gini', 'max_depth': 3, 'min_samples_leaf': 6}

Mean cross-validated score of the estimator: 0.695


Score computed on the test set: 0.154

----------------------------------------------------------------------------------------------------
Parameters set found:

{'class_weight': 'balanced', 'criterion': 'gini', 'max_depth': 7, 'min_samples_leaf': 2}

Mean cross-validated score of the estimator: 0.715


Score computed on the test set: 0.500

----------------------------------------------------------------------------------------------------
Parameters set found:

{'class_weight': 'balanced', 'criterion': 'entropy', 'max_depth': 7, 'min_samples_leaf': 4}

Mean cross-validated score of the estimator: 0.705


Score computed on the test set: 0.308

----------------------------------------------------------------------------------------------------
Parameters set found:

{'class_weight': 'balanced', 'crit

Parameters set found:

{'class_weight': 'balanced', 'criterion': 'gini', 'max_depth': 7, 'min_samples_leaf': 2}

Mean cross-validated score of the estimator: 0.693


Score computed on the test set: 0.250

----------------------------------------------------------------------------------------------------
Parameters set found:

{'class_weight': 'balanced', 'criterion': 'entropy', 'max_depth': 4, 'min_samples_leaf': 5}

Mean cross-validated score of the estimator: 0.645


Score computed on the test set: 0.364

----------------------------------------------------------------------------------------------------
Parameters set found:

{'class_weight': 'balanced', 'criterion': 'gini', 'max_depth': 3, 'min_samples_leaf': 6}

Mean cross-validated score of the estimator: 0.681


Score computed on the test set: 0.200

----------------------------------------------------------------------------------------------------
### ITERATION 7 ###


Parameters set found:

{'class_weight': 'balanced', 'crit

In [16]:
AUG(X_not_encoded, y, [1,2], tuned_parameters, random_state_tree = seed,
    scoring_func = accuracy_score, filename = '12-acc-smote', aug_method = 'smote')

### ITERATION 1 ###


Parameters set found:

{'class_weight': None, 'criterion': 'gini', 'max_depth': 7, 'min_samples_leaf': 2}

Mean cross-validated score of the estimator: 0.755


Score computed on the test set: 0.591

----------------------------------------------------------------------------------------------------
Parameters set found:

{'class_weight': None, 'criterion': 'entropy', 'max_depth': 5, 'min_samples_leaf': 2}

Mean cross-validated score of the estimator: 0.772


Score computed on the test set: 0.545

----------------------------------------------------------------------------------------------------
Parameters set found:

{'class_weight': 'balanced', 'criterion': 'entropy', 'max_depth': 8, 'min_samples_leaf': 3}

Mean cross-validated score of the estimator: 0.753


Score computed on the test set: 0.619

----------------------------------------------------------------------------------------------------
Parameters set found:

{'class_weight': None, 'criterion': 'entrop

Parameters set found:

{'class_weight': None, 'criterion': 'gini', 'max_depth': 5, 'min_samples_leaf': 2}

Mean cross-validated score of the estimator: 0.758


Score computed on the test set: 0.619

----------------------------------------------------------------------------------------------------
Parameters set found:

{'class_weight': None, 'criterion': 'entropy', 'max_depth': 5, 'min_samples_leaf': 4}

Mean cross-validated score of the estimator: 0.846


Score computed on the test set: 0.667

----------------------------------------------------------------------------------------------------
Parameters set found:

{'class_weight': 'balanced', 'criterion': 'gini', 'max_depth': 5, 'min_samples_leaf': 4}

Mean cross-validated score of the estimator: 0.735


Score computed on the test set: 0.571

----------------------------------------------------------------------------------------------------
### ITERATION 7 ###


Parameters set found:

{'class_weight': 'balanced', 'criterion': 'gin

In [17]:
AUG(X_not_encoded, y, [1,2], tuned_parameters, random_state_tree = seed,
    scoring_func = f1_score, filename = '12-f1-smote', aug_method = 'smote')

### ITERATION 1 ###


Parameters set found:

{'class_weight': 'balanced', 'criterion': 'gini', 'max_depth': 6, 'min_samples_leaf': 8}

Mean cross-validated score of the estimator: 0.658


Score computed on the test set: 0.500

----------------------------------------------------------------------------------------------------
Parameters set found:

{'class_weight': 'balanced', 'criterion': 'entropy', 'max_depth': 3, 'min_samples_leaf': 4}

Mean cross-validated score of the estimator: 0.607


Score computed on the test set: 0.222

----------------------------------------------------------------------------------------------------
Parameters set found:

{'class_weight': 'balanced', 'criterion': 'gini', 'max_depth': 6, 'min_samples_leaf': 2}

Mean cross-validated score of the estimator: 0.618


Score computed on the test set: 0.250

----------------------------------------------------------------------------------------------------
Parameters set found:

{'class_weight': 'balanced', 'crit

Parameters set found:

{'class_weight': 'balanced', 'criterion': 'entropy', 'max_depth': 6, 'min_samples_leaf': 2}

Mean cross-validated score of the estimator: 0.595


Score computed on the test set: 0.286

----------------------------------------------------------------------------------------------------
Parameters set found:

{'class_weight': None, 'criterion': 'gini', 'max_depth': 8, 'min_samples_leaf': 2}

Mean cross-validated score of the estimator: 0.710


Score computed on the test set: 0.444

----------------------------------------------------------------------------------------------------
Parameters set found:

{'class_weight': 'balanced', 'criterion': 'entropy', 'max_depth': 3, 'min_samples_leaf': 6}

Mean cross-validated score of the estimator: 0.715


Score computed on the test set: 0.615

----------------------------------------------------------------------------------------------------
### ITERATION 7 ###


Parameters set found:

{'class_weight': 'balanced', 'criteri